## Try #1

In [1]:
#https://medium.com/@bedigunjit/simple-guide-to-text-classification-nlp-using-svm-and-naive-bayes-with-python-421db3a72d34

import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from collections import Counter

#import dill

if __name__ == "__main__":
    
        # Reproduce the same result every time if the script is kept consistent otherwise each run will produce different results
        np.random.seed(500)
    
        #[1] Read the data
        Corpus = pd.read_json(r"C:\Users\Panos\Desktop\Dissert\Code\Sample_Video_Games_5.json", lines=True, encoding='latin-1')
        Corpus = Corpus[['reviewText','overall']]
        
        # Print some info
        Corpus.info()
        print(Corpus.overall.value_counts())
        
        #[1.5] Reduce number of classes
        for index,entry in enumerate(Corpus['overall']):
             if entry == 1.0 or entry == 2.0:
                 Corpus.loc[index,'overall_final'] = -1
             elif entry == 3.0:
                 Corpus.loc[index,'overall_final'] = 0
             elif entry == 4.0 or entry == 5.0:
                 Corpus.loc[index,'overall_final'] = 1
                
        #[2] Preprocessing
        
        # Step - a : Remove blank rows if any.
        Corpus['reviewText'].dropna(inplace=True)
        # Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
        Corpus['reviewText'] = [entry.lower() for entry in Corpus['reviewText']]
        # Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
        Corpus['reviewText'] = [word_tokenize(entry) for entry in Corpus['reviewText']]
            
        #Print the first 3 rows
        print(Corpus.iloc[:3])
        print("hey yo")
        
        #dill.dump_session('notebook_env.db')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
reviewText    100 non-null object
overall       100 non-null int64
dtypes: int64(1), object(1)
memory usage: 1.7+ KB
5    53
4    17
1    15
3    10
2     5
Name: overall, dtype: int64
                                          reviewText  overall  overall_final
0  [installing, the, game, was, a, struggle, (, b...        1           -1.0
1  [if, you, like, rally, cars, get, this, game, ...        4            1.0
2  [1st, shipment, received, a, book, instead, of...        1           -1.0
hey yo


In [12]:
        #[3] Prepare Train and Test Data sets
            
        Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['reviewText'],Corpus['overall_final'],test_size=0.3)
        
        print(Counter(Train_Y).values()) # counts the elements' frequency
        
        #[4] Encoding
        
        Encoder = LabelEncoder()
        Train_Y = Encoder.fit_transform(Train_Y)
        Test_Y = Encoder.fit_transform(Test_Y)
        
        print(Corpus['reviewText'][0])
        
#         #[5] Word Vectorization
        
#         Tfidf_vect = TfidfVectorizer(max_features=10000)
#         Test_X_Tfidf = Tfidf_vect.fit_transform(Corpus['reviewText'])
#         Train_X_Tfidf = Tfidf_vect.transform(Train_X)
#         Test_X_Tfidf = Tfidf_vect.transform(Test_X)

dict_values([53, 12, 5])
['installing', 'the', 'game', 'was', 'a', 'struggle', '(', 'because', 'of', 'games', 'for', 'windows', 'live', 'bugs', ')', '.some', 'championship', 'races', 'and', 'cars', 'can', 'only', 'be', '``', 'unlocked', "''", 'by', 'buying', 'them', 'as', 'an', 'addon', 'to', 'the', 'game', '.', 'i', 'paid', 'nearly', '30', 'dollars', 'when', 'the', 'game', 'was', 'new', '.', 'i', 'do', "n't", 'like', 'the', 'idea', 'that', 'i', 'have', 'to', 'keep', 'paying', 'to', 'keep', 'playing.i', 'noticed', 'no', 'improvement', 'in', 'the', 'physics', 'or', 'graphics', 'compared', 'to', 'dirt', '2.i', 'tossed', 'it', 'in', 'the', 'garbage', 'and', 'vowed', 'never', 'to', 'buy', 'another', 'codemasters', 'game', '.', 'i', "'m", 'really', 'tired', 'of', 'arcade', 'style', 'rally/racing', 'games', 'anyway.i', "'ll", 'continue', 'to', 'get', 'my', 'fix', 'from', 'richard', 'burns', 'rally', ',', 'and', 'you', 'should', 'to', '.', ':', ')', 'http', ':', '//www.amazon.com/richard-burn

In [ ]:
        #[6] SMOTE (Synthetic Minority Over-Sampling Technique)
        from imblearn.under_sampling import NearMiss, RandomUnderSampler
        
        nm = NearMiss(ratio='not minority',random_state=777, version=1, n_neighbors=1)
        X_nm, y_nm = nm.fit_sample(Train_X_Tfidf, Train_Y)
        
        print(Counter(y_nm).values()) # counts the elements' frequency

In [ ]:
        #the vocabulary that it has learned from the corpus
        print(Tfidf_vect.vocabulary_)
        
        #the vectorized data
        print(Train_X_Tfidf)

In [ ]:
        #[7] Use the ML Algorithms to Predict the outcome
        
        # fit the training dataset on the NB classifier
        Naive = naive_bayes.MultinomialNB()
        Naive.fit(X_nm,y_nm)
        # predict the labels on validation dataset
        predictions_NB = Naive.predict(Test_X_Tfidf)
        # Use accuracy_score function to get the accuracy
        print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)
        # Making the confusion matrix
        from sklearn.metrics import confusion_matrix
        cm = confusion_matrix(Test_Y, predictions_NB)
        print("-----------------cm------------------")
        print(cm)
        print("-------------------------------------")
        
        #[8] Support Vector Machine
        
        # Classifier - Algorithm - SVM
        # fit the training dataset on the classifier
        SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
        SVM.fit(X_nm,y_nm)
        # predict the labels on validation dataset
        predictions_SVM = SVM.predict(Test_X_Tfidf)
        # Use accuracy_score function to get the accuracy
        print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)